In [466]:
import math
import random
import pandas as pd
import numpy as np

from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import Grid
from mesa.time import RandomActivation
from mesa.batchrunner import BatchRunner

In [1258]:
def get_mining_summary(model):
    try:
        df = model.blockchain.mempool
        df['mining_time'] = df['block_mined'] - df['block_submitted']

        return df['mining_time'].astype(int).describe() 
    except:
        return "Failed on tick " + str(model.schedule.steps) 

In [990]:
def get_agent_expiry_ticks(model):
    # In analysis will need to check expiry ticks column and 
    # cast to int to get summary stats.
    
    s = pd.DataFrame(columns=['unique_id', 'expiry_tick'])
    
    for a in model.schedule.agents:
        s = s.append({'unique_id': int(a.unique_id), 'expiry_tick': a.dead}, ignore_index=True)
    
    return s

In [645]:
def get_informational_currency(agent, window=1):
    """
    Calculate the proportion of data collected by a 
    sensor that has been committed to the chain.
    
    **Note: This counts each tick as equal, even if 
    different volumes of data were collected during
    the ticks.**
    
    Attributes:
        agent: The Sensor for which to calculate 
            informational currency measure.
        window: The window through which to measure. 
        
            If window = 1, check proportion of data 
            reflected on chain from all time.
            
            If window > 1, check proportion of data in 
            most recent n=window ticks that is reflected 
            on chain
    """
    
    model = agent.model
    agent_id = agent.unique_id
    
    if window == 1 or window > len(model.blockchain.chain):
        try:
            return model.blockchain.chain[agent_id].value_counts()[True] / len(model.blockchain.chain[agent_id])
        except KeyError:
            return 0
    
    else:
        df = model.blockchain.chain[window * -1 :] # Last n rows
        try:
            return df[agent_id].value_counts()[True] / len(df[agent_id])
        except KeyError:
            return 0


In [859]:
def create_tx_dict(from_address, nonce, start, end, gas_price, num_bytes):
    """
    Assemble variables into a dictionary representing 
    an (abstracted) Ethereum transaction.
    """
    
    tx = {}
        
    tx['nonce'] = nonce
    tx['from_address'] = from_address # << Sensor ID
    tx['start_sync'] = start
    tx['end_sync'] = end
    tx['gas_price'] = gas_price
    # self.gas_limit = gas_limit # << unused parameter
    tx['num_bytes'] = num_bytes

    return tx

In [806]:
class Sensor(Agent):
    """
    An edge device. 
    
    Attributes:
        battery_life: Total battery on board.
        record_cost, record_freq, record_bytes: Values
            to calculate sensor recording behavior
        compute_cost_per_byte: An abstracted link between
            bytes computed and energy consumed
        info_reduction: An abstracted scaling metric to 
            calculate the amount of data to transmit based
            on the amount of data collected at the edge. 
            Represents edge computing capability.
        sign_cost: An estimate of the energy consumed to sign a tx. 
        transmit_cost_per_byte: A link between bytes transmitted
            and energy consumed
        transmit_freq: How often data is transmitted from the sensor
            to the blockchain. If n > 1, every n ticks. If n < 1, 
            n is the probability of transmitting each tick
        gas_price: The gas_price a sensor is willing to pay to get 
            its transactions mined by the EVM.
        blockchain: a pointer to the blockchain each sensor is 
            connecting to, so it can invoke class methods
        stochasticity: The measure of randomness, applied when 
            appropriate, to simulate variation
        model: The model that is creating the sensor.
        
    """
    def __init__(self, unique_id, battery_life, 
                record_cost, record_freq, record_bytes, 
                compute_cost_per_byte, info_reduction,
                sign_cost,
                transmit_cost_per_byte, transmit_freq,
                gas_price, blockchain, stochasticity,
                model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print('Creating Sensor agent ID', unique_id)
           
        self.battery_life = battery_life
        self.dead = False
        self.record_cost = record_cost
        self.record_freq = record_freq
        self.record_bytes = record_bytes
        self.compute_cost_per_byte = compute_cost_per_byte
        self.info_reduction = info_reduction
        self.sign_cost = sign_cost
        self.transmit_cost_per_byte = transmit_cost_per_byte
        self.transmit_freq = transmit_freq
        self.gas_price = gas_price
        self.blockchain = blockchain
        self.stochasticity = stochasticity
        
        self.gwei_spent = 0
        self.data_collected = 0
        self.last_sync = 0
        self.nonce = 0
        self.db = np.array([]) # << bytes recorded per tick
        
        self.blockchain.chain.loc[0, self.unique_id] = False

    
    def record(self):
        if (self.record_freq > 1 and self.model.schedule.steps % self.record_freq == 0) or self.record_freq > random.random():

                self.battery_life -= self.record_cost # << independent of number of bytes? 
                new_record = self.record_bytes
                if self.stochasticity:
                    new_record = math.ceil(new_record  * (1 + random.uniform(self.stochasticity * -1, self.stochasticity)))

                self.db = np.append(self.db, new_record)

        else:
            self.db = np.append(self.db, 0)

            
    def next_nonce(self):
        self.nonce += 1
        return self.nonce
   

    def transmit(self):
        
        if self.model.verbose:
            print("Transmitting tx from", self.unique_id)

        # Prepare data for transmission:
        
        # Calculate number of bytes to transmit (result of edge computation)
        bytes_collected = np.sum(self.db[self.last_sync : ])
        num_bytes_to_transmit = self.compute(bytes_collected)
        
        if num_bytes_to_transmit > 0:

            tx = create_tx_dict(self.unique_id, self.next_nonce(), 
                           self.last_sync, self.model.schedule.steps, 
                           self.gas_price, num_bytes_to_transmit)

            # Prepare and sign tx
            self.sign()

            # Transmit, subtracting energy cost and adding gwei cost
            self.battery_life -= self.transmit_cost_per_byte * num_bytes_to_transmit
            self.blockchain.add_to_mempool(tx)

            self.last_sync = self.model.schedule.steps
    
    
    def compute(self, num_bytes):
        
        # Only invoked from within the transmit() method
        
        if self.info_reduction is not 1:
            self.battery_life -= self.compute_cost_per_byte * num_bytes
            return math.ceil(self.info_reduction * num_bytes)
        else:
            return num_bytes
            # with no compute cost
    
    
    def sign(self):
        
        # Only invoked from within the transmit() method
        self.battery_life -= self.sign_cost
    
    
    def confirm_tx(self, tx):
        
        # Here we could include learning element ...

#         mine_lag =  self.model.schedule.steps - tx.block_submitted
        # if mine_lag took too long - or was too quick - according to some threshold:
        #      change transmit_freq, info_reduction or gas_price ...
        
        self.gwei_spent += tx.gas_spend
    
    
    def total_data_collected(self):
        return self.db.sum()
    
    
    def step(self):
        
        if not self.dead:   
            if self.battery_life < 0 and self.battery_life is not -999:
                if self.model.verbose:
                    print("Sensor", self.unique_id, "out of battery at tick", self.model.schedule.steps)
                self.dead = self.model.schedule.steps

            self.record()    

            if self.transmit_freq >= 1:
                if self.model.schedule.steps % self.transmit_freq == 0:
                    self.transmit()
            elif self.transmit_freq > random.random():
                self.transmit()



In [981]:
class Blockchain(Agent):
    
    def __init__(self, unique_id, gas_price, block_gas_limit,
                gas_per_byte, gas_per_second, avg_block_time, model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print("Blockchain created: ID", unique_id)
        
#         self.unique_id = unique_id
        self.gas_price = gas_price
        self.block_gas_limit = block_gas_limit
        self.gas_per_byte = gas_per_byte
        self.gas_per_second = gas_per_second
        self.avg_block_time = avg_block_time
        self.chain = pd.DataFrame()
        
        self.tx_ct = 0
        self.mempool = pd.DataFrame(columns=["from_address", "nonce", 
                                             "start_sync", "end_sync", 
                                             "gas_price", "num_bytes", 
                                             "gas_spend", "tx_id",
                                             "mined", "block_submitted"])

    def add_to_mempool(self, tx):
        tx['gas_spend'] = tx['gas_price'] * self.gas_per_byte * tx['num_bytes']

        tx['tx_id'] = self.tx_ct
        tx['mined'] = False
        tx['block_submitted'] = self.model.schedule.steps
        row = pd.DataFrame(tx, index = [self.tx_ct])
        
        self.tx_ct += 1
        self.mempool = self.mempool.append(row, ignore_index=True)
    
    def write_data(self, num_bytes):

        gwei_spent = self.gas_per_byte * num_bytes * self.gas_price
        return gwei_spent

    def mine_block(self):
        
        if self.model.verbose:
            print("Mining BLOCK NUMBER:", self.model.schedule.steps)
        self.chain.loc[self.model.schedule.steps] = [False for col in self.chain.columns]

        # Sort mempool to get highest-value transactions
        mp = self.mempool[self.mempool['mined'] == False].sort_values(by=['gas_spend', 'block_submitted']).reset_index()
        
        if len(mp) > 0:
            
            mp['cum_gas'] = mp['gas_spend'].cumsum()     
            if mp['cum_gas'].max() > self.block_gas_limit:
                
                # If we cannot include all transactions in a block, fit as many as possible ...
                tx_mined = mp[0 : mp[mp['cum_gas'] > self.block_gas_limit].index[0] - 1]
                
            else:
                tx_mined = mp[0 : ]
            
            if self.model.verbose:
                print("Mining", len(tx_mined), "out of", len(mp), "unvalidated transactions.\n")
                print("Gas value:", tx_mined['gas_spend'].sum(), '\n')
            
            for tx in tx_mined.iterrows():
                
                if self.model.verbose:
                    print("Mining tx id:", tx[1].tx_id)
                    
                self.model.schedule._agents[tx[1].from_address].confirm_tx(tx[1])
                self.mempool.loc[self.mempool['tx_id'] == tx[1].tx_id, "mined"] = True
                self.mempool.loc[self.mempool['tx_id'] == tx[1].tx_id, "block_mined"] = self.model.schedule.steps
                
                self.chain.loc[tx[1].start_sync : tx[1].end_sync, tx[1].from_address] = True
                
        else:
            if self.model.verbose:
                print('Empty mempool')
            pass
        
        if self.model.verbose:
            print('\n')
    
    # Not used:
    def compute(self, num_seconds):
        gwei_spent = self.gas_per_second * num_seconds * self.gas_price
        return gwei_spent
    

In [977]:
class SensorBlockchainNetwork(Model):
    
    def __init__(self, num_sensors,
                 stochasticity=0,
       # Blockchain vars:
                 blockchain_gas_price=20,
                 block_gas_limit=9000000,
                 gas_per_byte=625,
                 gas_per_second=75000000,
                 avg_block_time=13,
                    # gas_per_byte and gas_per_second calculated based on 
                    # https://hackernoon.com/ether-purchase-power-df40a38c5a2f
                
        # Sensor vars:
                 battery_life=1000,
                 record_cost=1, record_freq=1, record_bytes=32,
                 compute_cost_per_byte=1, info_reduction=1,
                 sign_cost=0.1, 
                 transmit_cost_per_byte=1, transmit_freq=1,
                 sensor_gas_price=20,
                 
                 # Model vars:
                 verbose=False):
        
        super().__init__()
        
        
        self.verbose = verbose
        if self.verbose:
            print('Verbose model')
        
        self.running = True
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
                                model_reporters = {
                                    "active_sensors": lambda m: len([a for a in m.schedule.agents if not a.dead]),
                                    # chain size
                                    # information availability   
                                },
                                agent_reporters = {
                                    "gwei_spent": lambda a: a.gwei_spent,
                                    "battery_life": lambda a: a.battery_life,
                                    "data_collected": lambda a: a.total_data_collected(),
                                    "informational_currency": lambda a: get_informational_currency(a, 20)
                                })
        
        self.blockchain = Blockchain(self.next_id(), blockchain_gas_price, block_gas_limit,
                                    gas_per_byte, gas_per_second, avg_block_time, self) 

        
        for i in range(num_sensors):
            sensor = Sensor(self.next_id(), battery_life, 
                            record_cost, record_freq, record_bytes,
                            compute_cost_per_byte, info_reduction, 
                            sign_cost,
                            transmit_cost_per_byte, transmit_freq, 
                            sensor_gas_price,
                            self.blockchain, stochasticity, # << each sensor gets the same amount of stochasticity?
                            self)
            
            self.schedule.add(sensor)
    
        # Mine genesis block
        self.blockchain.chain.loc[1] = [False for col in self.blockchain.chain.columns]

        if self.verbose:
            print(num_sensors, "instantiated and added to schedule.")
    
    def step(self):
        self.schedule.step()
        if self.schedule.steps > 1:
            self.blockchain.mine_block()
        self.datacollector.collect(self)

In [923]:
model = SensorBlockchainNetwork(20, stochasticity=0.5, record_freq=0.07, record_bytes=140, battery_life=10000, verbose=False)

In [926]:
for i in range(500):
    model.step()

Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty mempool
Empty 

In [891]:
a_df = model.datacollector.get_agent_vars_dataframe()
m_df = model.datacollector.get_model_vars_dataframe()

In [897]:
model.schedule._agents[2].dead

34

# Model Execution and Analysis

Here we instantiate a mesa BatchRunner object, which contains methods for iterating through the arrays of variable parameters we pass in and executing a model for each arrangement. As a result we can access a pandas dataframe that contains all the agent, model tick and model run variables, as defined in the SensorBlockchainNetwork class definition, and in the model_reporters parameter passed into the BatchRunner object instantiation. 

We pass this dataframe into the following functions to extract relevant data and metrics, to be analyzed and visualized to discern system dynamic across the range of parameters swept. 

In [1296]:
def extract_mining_summary_from_batch_df(batch_df):
    """
    A function extracting mining descriptive statistics
    from the dataframe returned from the 
    batchrunner.get_model_vars_dataframe() method invocation
    """
    df = pd.DataFrame(columns=['num_sensors',  'record_freq', 'record_bytes', 'stochasticity', 'mean_mining_time', 'std', 'min', 'max'])

    for i, row in batch_df.iterrows():

        df.loc[i, 'record_freq'] = row['record_freq']
        df.loc[i, 'num_sensors'] = row['num_sensors']
        df.loc[i, 'record_freq'] = row['record_freq']
        df.loc[i, 'record_bytes'] = row['record_bytes']
        df.loc[i, 'stochasticity'] = row['stochasticity']

        try:
            df.loc[i, 'mean_mining_time'] = row['mining_summary']['mean']
            df.loc[i, 'std'] = row['mining_summary']['std']
            df.loc[i, 'min'] = row['mining_summary']['min']
            df.loc[i, 'max'] = row['mining_summary']['max']
        except:
            df.loc[i, 'mean_mining_time'] = row['mining_summary']
    
    return df

In [1268]:
def extract_gwei_spent_from_batch_df(batch_df):
    df_tmp = pd.DataFrame(columns=['num_sensors',  'record_freq', 'record_bytes', 'stochasticity'])

    for i, row in batch_df.iterrows():
        df_tmp.loc[i, 'num_sensors'] = row['num_sensors']
        df_tmp.loc[i, 'record_freq'] = row['record_freq']
        df_tmp.loc[i, 'record_bytes'] = row['record_bytes']
        df_tmp.loc[i, 'stochasticity'] = row['stochasticity']

        df_gwei = row['data_collector'].get_agent_vars_dataframe()['gwei_spent'].unstack()

        for col in df_gwei.columns:
            df_gwei[str(col) + '_shift'] = df_gwei[col].shift(1)
            df_gwei[str(col) + '_gwei_spend'] = df_gwei[col] - df_gwei[str(col) + '_shift']

        df_gwei = df_gwei[[col for col in df_gwei.columns if 'spend' in str(col)]]

        df_tmp.loc[i, 'df_gwei_spent'] = InfoCurrency(df_gwei)

    return df_tmp

In [1204]:
class InfoCurrency:
    """ A cheeky workaround to store dataframes as a value in another dataframe ..."""
    def __init__(self, df):
        self.df = df

def extract_informational_currency_from_batch_df(batch_df):
    
    df_tmp = pd.DataFrame(columns=['num_sensors',  'record_freq', 'record_bytes', 'stochasticity', 'info_currency_df'])
    for i, row in batch_df.iterrows():
        df_tmp.loc[i, 'num_sensors'] = row['num_sensors']
        df_tmp.loc[i, 'record_freq'] = row['record_freq']
        df_tmp.loc[i, 'record_bytes'] = row['record_bytes']
        df_tmp.loc[i, 'stochasticity'] = row['stochasticity']

        df_tmp.loc[i, 'info_currency_df'] = InfoCurrency(row['data_collector'].get_agent_vars_dataframe()['informational_currency'].unstack())
    
    return df_tmp

In [1241]:
def extract_network_life_cycle_from_batch_df(batch_df):
    return batch_df[[ 'num_sensors','record_bytes', 'record_freq', 'stochasticity', 'agent_expiry_ticks']]

In [1253]:
fixed_params = {
    "blockchain_gas_price": 20,
    "block_gas_limit": 8000000,
    "gas_per_byte": 625,
    "gas_per_second": 75000000,
    "avg_block_time": 13,
    "verbose": False,
    "battery_life": 10000,
    
    "num_sensors": 10,
    "stochasticity": 0
}

variable_params = {
    "num_sensors": range(1, 110, 10),
    "stochasticity": np.arange(0, 1, 0.1),
    "record_bytes": [1, 10, 100],
    "record_freq": [0.001, 0.01, 0.1, 1]
}

In [1259]:
# Batch Run

batch_runner = BatchRunner(
    SensorBlockchainNetwork,
    variable_params,
    fixed_params,
    iterations = 1,
    max_steps = 300,
    model_reporters = {"data_collector": lambda m: m.datacollector,
                      "agent_expiry_ticks": get_agent_expiry_ticks,
                      "mining_summary": get_mining_summary}
)

In [1260]:
batch_runner.run_all()

1320it [2:54:47,  7.26s/it]


In [1261]:
batch_df = batch_runner.get_model_vars_dataframe()

In [1274]:
batch_df.loc[0]['mining_summary']

count    2.0
mean     1.0
std      0.0
min      1.0
25%      1.0
50%      1.0
75%      1.0
max      1.0
Name: mining_time, dtype: float64

In [1297]:
mining_summary = extract_mining_summary_from_batch_df(batch_df)


In [1265]:
info_currency_summary = extract_informational_currency_from_batch_df(batch_df)


In [1266]:
network_life_cycle_summary = extract_informational_currency_from_batch_df(batch_df)


In [1269]:
gwei_spent_summary = extract_gwei_spent_from_batch_df(batch_df)

In [1300]:
mining_summary.to_pickle('./data/run-1/mining-summary.pkl')
info_currency_summary.to_pickle('./data/run-1/info-currency-summary.pkl')
network_life_cycle_summary.to_pickle('./data/run-1/network-life-cycle-summary.pkl')
gwei_spent_summary.to_pickle('./data/run-1/gwei-spent-summary.pkl')


In [1257]:
model.blockchain.mempool

,block_mined,block_submitted,end_sync,from_address,gas_price,gas_spend,mined,nonce,num_bytes,start_sync,tx_id,mining_time
0,2.0,0,0,14,20,1412500.0,True,1,113.0,0,0,2
1,2.0,0,0,9,20,2300000.0,True,1,184.0,0,1,2
2,2.0,1,1,9,20,2300000.0,True,2,184.0,0,2,1
3,2.0,1,1,14,20,2387500.0,True,2,191.0,0,3,1
4,3.0,2,2,14,20,975000.0,True,3,78.0,1,4,1
5,4.0,3,3,12,20,2212500.0,True,1,177.0,0,5,1
6,5.0,4,4,4,20,1350000.0,True,1,108.0,0,6,1
7,5.0,4,4,16,20,2587500.0,True,1,207.0,0,7,1
8,5.0,4,4,12,20,2212500.0,True,2,177.0,3,8,1
9,6.0,5,5,3,20,2287500.0,True,1,183.0,0,9,1


In [1187]:
batch_df

,record_bytes,record_freq,Run,agent_expiry_ticks,data_collector,mining_summary,blockchain_gas_price,block_gas_limit,gas_per_byte,gas_per_second,avg_block_time,verbose,battery_life,num_sensors,stochasticity
0,1,0.001,0,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 8.0 mean 1.0 std 0.0 min ...,20,8000000,625,75000000,13,False,10000,10,0
1,1,0.010,1,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 54.0 mean 1.0 std 0.0 min ...,20,8000000,625,75000000,13,False,10000,10,0
2,1,0.100,2,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 515.000000 mean 1.001942 std ...,20,8000000,625,75000000,13,False,10000,10,0
3,1,1.000,3,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 3000.000000 mean 1.003333 std ...,20,8000000,625,75000000,13,False,10000,10,0
4,10,0.001,4,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 8.0 mean 1.0 std 0.0 min ...,20,8000000,625,75000000,13,False,10000,10,0
5,10,0.010,5,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 50.0 mean 1.0 std 0.0 min ...,20,8000000,625,75000000,13,False,10000,10,0
6,10,0.100,6,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 573.000000 mean 1.006981 std ...,20,8000000,625,75000000,13,False,10000,10,0
7,10,1.000,7,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 3000.000000 mean 1.003333 std ...,20,8000000,625,75000000,13,False,10000,10,0
8,100,0.001,8,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 2.0 mean 1.0 std 0.0 min ...,20,8000000,625,75000000,13,False,10000,10,0
9,100,0.010,9,unique_id expiry_tick 0 2 Fals...,<mesa.datacollection.DataCollector object at 0...,count 62.0 mean 1.0 std 0.0 min ...,20,8000000,625,75000000,13,False,10000,10,0


In [1170]:
extract_mining_summary_from_batch_df(batch_df)

,num_sensors,record_freq,record_bytes,stochasticity,mean_mining_time,std,min,max
0,10,0.001,1,0,1,0,1,1
1,10,0.01,1,0,1,0,1,1
2,10,0.1,1,0,1.00523,0.0721681,1,2
3,10,0.001,10,0,1,0,1,1
4,10,0.01,10,0,1,0,1,1
5,10,0.1,10,0,1.00174,0.0416667,1,2
6,10,0.001,100,0,1,0,1,1
7,10,0.01,100,0,1,0,1,1
8,10,0.1,100,0,1.00333,0.0576389,1,2


In [1022]:
agent_df = batch_df['data_collector'][0].get_agent_vars_dataframe()
model_df = batch_df['data_collector'][0].get_model_vars_dataframe()

In [1030]:
batch_df['mining_summary'][6]

count    1560.000000
mean       10.114103
std         5.018611
min         1.000000
25%         6.000000
50%        10.000000
75%        14.000000
max        19.000000
Name: mining_time, dtype: float64

In [1035]:
mining_df = pd.DataFrame(columns=batch_df['mining_summary'][0].index)

In [1138]:
df_t = batch_df['data_collector'][0].get_agent_vars_dataframe()

In [1139]:
df_t = df_t.unstack()['gwei_spent']

In [1230]:
df_tmp.loc[0, 'df_gwei_spent'].df

AgentID,2_gwei_spend,3_gwei_spend,4_gwei_spend,5_gwei_spend,6_gwei_spend,7_gwei_spend,8_gwei_spend,9_gwei_spend,10_gwei_spend,11_gwei_spend
Step,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1225]:
row = batch_df.loc[0]

df_tmp.loc[i, 'num_sensors'] = row['num_sensors']
df_tmp.loc[i, 'record_freq'] = row['record_freq']
df_tmp.loc[i, 'record_bytes'] = row['record_bytes']
df_tmp.loc[i, 'stochasticity'] = row['stochasticity']

df_gwei = row['data_collector'].get_agent_vars_dataframe()

# for col in df_gwei.columns:
#     df_gwei[str(col) + '_shift'] = df_gwei[col].shift(1)
#     df_gwei[str(col) + '_gwei_spend'] = df_gwei[col] - df_gwei[str(col) + '_shift']

# df_gwei = df_gwei[[col for col in df_gwei.columns if 'spend' in str(col)]]

# df_tmp.loc[i, 'df_gwei_spent'] = InfoCurrency(df_gwei)

In [1228]:
df_gwei['gwei_spent'].unstack()

AgentID,2,3,4,5,6,7,8,9,10,11
Step,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1102]:
df_t['lag'] = df_t[5] - df_t['5_shift']

In [1104]:
df_t[df_t['lag'] > 0]

AgentID,2,3,4,5,6,7,8,9,10,11,5_shift,lag
Step,,,,,,,,,,,,
179,0.0,0.0,0.0,400000.0,0.0,0.0,0.0,800000.0,800000.0,0.0,0.0,400000.0
180,0.0,0.0,0.0,800000.0,0.0,0.0,0.0,800000.0,800000.0,0.0,400000.0,400000.0


In [1068]:
mining_df = extract_mining_summary_from_series(batch_df)

TypeError: string indices must be integers

In [1069]:
df_holder = pd.DataFrame(columns=['record_freq', 'mean', 'std', 'min', 'max'])


In [1017]:
info_df.describe()

AgentID,2,3,4,5,6,7,8,9,10,11,informational_currency
count,300.0,300.000000,300.000000,300.000000,300.000000,300.000000,300.0000,300.0,300.000000,300.000000,300.000000
mean,0.0,0.066167,0.151084,0.099500,0.139333,0.069667,0.1365,0.0,0.211667,0.069667,0.094358
std,0.0,0.205396,0.286746,0.245273,0.274233,0.206086,0.2789,0.0,0.316303,0.206086,0.066627
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000
25%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.045000
50%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.090000
75%,0.0,0.000000,0.150000,0.000000,0.100000,0.000000,0.0000,0.0,0.400000,0.000000,0.140000
max,0.0,0.950000,0.950000,0.950000,0.950000,0.950000,0.9500,0.0,0.950000,0.950000,0.280000


count    1374.000000
mean        2.015284
std         4.840552
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        71.000000
Name: mining_time, dtype: float64